In [ ]:
# Sam Brown, Zachary Katz
# Goal: Troubleshoot time for pulling tide data

import pyTMD

import my_lib.funcs

import util.coordinate_transforms

import Tides

import time

import pandas as pd
import numpy as np
import datetime

In [ ]:
# Set path to event files
evts_path = "/Users/sambrown04/Documents/SURF/Events/2013_2013Events2stas" 

# load events in
events_list = my_lib.funcs.load_evt(evts_path)

In [4]:
feat = my_lib.funcs.extract_event_features(events_list)

In [5]:
# first let us make a list of events with only gz stations
gz_df = pd.DataFrame(columns = ["station", "pre-slip_area", "slip_severity", "peak_time", "total_delta", "start_time"])

for i, event in enumerate(feat):

    for _,row in event.iterrows():

        # print(row['station'][0:2])
        if row['station'][0:2] == "gz":
            
            gz_df.loc[len(gz_df)] = row

In [6]:
# get average x and y cor and lat lon for the gz stations
# need a list of x cors and a list of y cors
x_cors = []
y_cors = []

pre = 'gz'
numbers = ['01','02','03','04','05','12','13','14','15','16','17','18'] # had to change as data frame did not have some of the stations?
    
for num in numbers:
    station = f"{pre}{num}"

    x_col = f"{station}x"
    y_col = f"{station}y"
    
    # loop through events to get first instance station is transmitting
    for i, event in enumerate(events_list):
        if not event[x_col].isna().any():# make sure location is transmitting
            # get first instance of coordinates
            x_cors.append(event.at[0, x_col]) 
            y_cors.append(event.at[0,y_col])
            break

In [7]:
avg_x_cor = sum(x_cors) / len(x_cors)
avg_y_cor = sum(y_cors) / len(y_cors)

In [ ]:
# Setup model
# Suspect this line creating the model is causing the long runtime. We are defining it here and passing it into the tidal_elevation function

### USER DEFINED PATH TO TIDE MODEL ###
tide_dir = "/Users/sambrown04/Documents/SURF"
#######################################

tide_mod = "CATS2008-v2023"

tides = Tides.Tide(tide_mod, tide_dir)

model1 = pyTMD.io.model(tide_dir, format="netcdf").elevation(tide_mod)
constituents = pyTMD.io.OTIS.read_constants(
    model1.grid_file,
    model1.model_file,
    model1.projection,
    type=model1.type,
    grid=model1.format,
)

In [26]:
# function that takes the x_cors and y_cors and time and returns the tide.


def get_tide_height(days, x_cor, y_cor, start_time):
    """ Get tide height for <days> days from initial date <start_time>, at the coordinates <x_cor> and <y_cor>.

    Parameters
    ----------
    days : int
        Number of days to calculate tide height for.
    x_cor : float
        PS71 x coordinate of tide calculation
    y_cor : _type_
        PS71 y coordinate of tide calculation
    start_time : _type_
        Starting date in %Y-%m-%d %H:%M:%S format

    Returns
    -------
    list[float]
        Tide heights
    """

    spacing = 1

    HR_PER_DAY = 24
    MIN_PER_HR = 60

    dates_timeseries = []
    initial_time = datetime.datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
    for i in range(days * HR_PER_DAY * MIN_PER_HR // spacing):  # 30 days * 24 hr/day * 60 min/hr * 1/10 calculations/min
        dates_timeseries.append(initial_time + datetime.timedelta(minutes=spacing * i))

    #convert to lon and lat
    lon, lat = util.coordinate_transforms.xy2ll(x_cor, y_cor)
    # print(lon, lat)

    
    tides = Tides.Tide(tide_mod, tide_dir)
    
    start_time = time.time()
    tide_results = tides.tidal_elevation(
        model1,
        constituents,
        [lon],
        [lat],
        dates_timeseries,
    ).data.T[0]
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    
    return tide_results
    
    
    
    #HR_PER_DAY * MIN_PER_HR // spacing
    

In [27]:
# Line profiler Jupyter extension - outputs how long each individual line of code takes to run
%load_ext line_profiler

%lprun -f get_tide_height get_tide_height(1, avg_x_cor, avg_y_cor, '2013-05-19 07:37:15')

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
Elapsed time: 38.12063145637512 seconds


Timer unit: 1e-09 s

Total time: 38.1235 s
File: /tmp/ipykernel_33510/365884760.py
Function: get_tide_height at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def get_tide_height(days, x_cor, y_cor, start_time):
     5                                               """ Get tide height for <days> days from initial date <start_time>, at the coordinates <x_cor> and <y_cor>.
     6                                           
     7                                               Parameters
     8                                               ----------
     9                                               days : int
    10                                                   Number of days to calculate tide height for.
    11                                               x_cor : float
    12                                                   PS71 x coordinate of tide calculation
    13                                          

In [30]:
# Time functions using the iPython magic time command
%time get_tide_height(1, avg_x_cor, avg_y_cor, '2013-05-19 07:37:15')
%time get_tide_height(365, avg_x_cor, avg_y_cor, '2013-05-19 07:37:15')

Elapsed time: 41.24638605117798 seconds
CPU times: user 38 s, sys: 3.2 s, total: 41.2 s
Wall time: 41.2 s
Elapsed time: 45.862751483917236 seconds
CPU times: user 46.7 s, sys: 3.35 s, total: 50 s
Wall time: 46.3 s


array([19.62466491, 19.37249167, 19.11957894, ..., 49.47227695,
       49.24247156, 49.01142783], shape=(525600,))